## This is the colab notebook which has some ways in which optimal portfolio can be made. The desgin overview of this portfolio is such that we have selected a combinations of 4 distinct portfolios from the set of 50 stocks

-- In the set of 50 stocks we are going to choose a set of 10 stocks in each portfolio

In [29]:
# Firstly we are going to load all the libraries which we need
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import cvxpy as cp
# importing the libraries



-- Now we are going to load the set of the nifty fifty stocks 

In [30]:
nifty50_Stocks = [
    "RELIANCE.NS", "TCS.NS", "INFY.NS", "HDFCBANK.NS", "ICICIBANK.NS", "KOTAKBANK.NS",
    "HINDUNILVR.NS", "SBIN.NS", "LT.NS", "ITC.NS", "AXISBANK.NS", "BAJFINANCE.NS",
    "ASIANPAINT.NS", "MARUTI.NS", "SUNPHARMA.NS", "ULTRACEMCO.NS", "NESTLEIND.NS", "WIPRO.NS",
    "NTPC.NS", "POWERGRID.NS", "COALINDIA.NS", "TITAN.NS", "TECHM.NS", "GRASIM.NS",
    "BHARTIARTL.NS", "HCLTECH.NS", "DRREDDY.NS", "ADANIENT.NS", "BAJAJFINSV.NS", "TATASTEEL.NS",
    "HDFCLIFE.NS", "SBILIFE.NS", "HEROMOTOCO.NS", "DIVISLAB.NS", "INDUSINDBK.NS", "JSWSTEEL.NS",
    "CIPLA.NS", "EICHERMOT.NS", "UPL.NS", "BPCL.NS", "BRITANNIA.NS", "SHREECEM.NS",
    "ONGC.NS", "APOLLOHOSP.NS", "BAJAJ-AUTO.NS", "M&M.NS", "HINDALCO.NS", "TATACONSUM.NS",
    "TATAMOTORS.NS", "SBICARD.NS"
]
#importing the nifty fifty stocks

# Defining the time for which we need the data for

data_stocks = yf.download(nifty50_Stocks, start="2014-01-01", end="2024-01-01")

print(data_stocks.head())

[*********************100%***********************]  50 of 50 completed


Price            Close                                                        \
Ticker     ADANIENT.NS APOLLOHOSP.NS ASIANPAINT.NS AXISBANK.NS BAJAJ-AUTO.NS   
Date                                                                           
2014-01-01   37.933193    927.892761    459.297485  247.552414   1448.022827   
2014-01-02   36.041821    890.798218    446.246918  243.740112   1434.920288   
2014-01-03   35.053799    911.283997    449.877106  241.556152   1431.521729   
2014-01-06   35.576042    905.301025    450.933960  240.674896   1426.046509   
2014-01-07   34.150455    892.281982    450.290741  236.450699   1427.292358   

Price                                                                         \
Ticker     BAJAJFINSV.NS BAJFINANCE.NS BHARTIARTL.NS    BPCL.NS BRITANNIA.NS   
Date                                                                           
2014-01-01     75.462006    151.039185    286.472443  31.349646   399.551483   
2014-01-02     72.309883    150.933380 

In [76]:
# Now we need to remove the missing data in the stocks 
data_stocks = data_stocks.dropna(axis=1, thresh=len(data_stocks)*0.95)
import random

# These stocks are diversified acorss different sectors to ensure no single sector dominates your profile
-- After this we are going to find the sectors of each of the following stocks which is available in the yfinance library

# What is stocks diversification
-- Stock diversification is the porcess of spreading your investments across different assests to reduce the overeall risks

-- What does Diversity do in the stock selection,it reduces the risk of getting a major loss when the market crashes as you dont have all your eggs in one basket

In [77]:
available_stocks = list(data_stocks .columns)
random.shuffle(available_stocks)


In [ ]:
portfolios = {
    f"Portfolio_{i+1}": available_stocks[i*10:(i+1)*10]
    for i in range(4)
}

portfolios
# these are the portfolios we have used

{'Portfolio_1': [('Volume', 'TECHM.NS'),
  ('Volume', 'UPL.NS'),
  ('Open', 'NTPC.NS'),
  ('High', 'HCLTECH.NS'),
  ('Open', 'JSWSTEEL.NS'),
  ('Low', 'EICHERMOT.NS'),
  ('Volume', 'TATAMOTORS.NS'),
  ('Close', 'TATASTEEL.NS'),
  ('Open', 'NESTLEIND.NS'),
  ('Open', 'TATAMOTORS.NS')],
 'Portfolio_2': [('High', 'TCS.NS'),
  ('High', 'NTPC.NS'),
  ('Low', 'BAJAJ-AUTO.NS'),
  ('Low', 'HEROMOTOCO.NS'),
  ('Open', 'LT.NS'),
  ('Close', 'BAJAJ-AUTO.NS'),
  ('High', 'INFY.NS'),
  ('Open', 'TATASTEEL.NS'),
  ('Volume', 'MARUTI.NS'),
  ('Open', 'BPCL.NS')],
 'Portfolio_3': [('Open', 'BAJAJ-AUTO.NS'),
  ('Low', 'TITAN.NS'),
  ('Close', 'COALINDIA.NS'),
  ('Volume', 'ASIANPAINT.NS'),
  ('Low', 'BAJAJFINSV.NS'),
  ('High', 'SUNPHARMA.NS'),
  ('High', 'JSWSTEEL.NS'),
  ('Volume', 'BAJAJ-AUTO.NS'),
  ('Close', 'DIVISLAB.NS'),
  ('Low', 'GRASIM.NS')],
 'Portfolio_4': [('Low', 'NESTLEIND.NS'),
  ('Low', 'BPCL.NS'),
  ('Close', 'LT.NS'),
  ('High', 'EICHERMOT.NS'),
  ('High', 'ADANIENT.NS'),
  ('Close'

In [81]:
returns = data_stocks.pct_change().dropna()

In [87]:
# Create a DataFrame to display the selected portfolios
portfolio_df = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in portfolios.items()]))

print("🎯 Selected Stocks in Each Portfolio:\n")
print(portfolio_df)

🎯 Selected Stocks in Each Portfolio:

               Portfolio_1             Portfolio_2              Portfolio_3  \
0       (Volume, TECHM.NS)          (High, TCS.NS)    (Open, BAJAJ-AUTO.NS)   
1         (Volume, UPL.NS)         (High, NTPC.NS)          (Low, TITAN.NS)   
2          (Open, NTPC.NS)    (Low, BAJAJ-AUTO.NS)    (Close, COALINDIA.NS)   
3       (High, HCLTECH.NS)    (Low, HEROMOTOCO.NS)  (Volume, ASIANPAINT.NS)   
4      (Open, JSWSTEEL.NS)           (Open, LT.NS)     (Low, BAJAJFINSV.NS)   
5      (Low, EICHERMOT.NS)  (Close, BAJAJ-AUTO.NS)     (High, SUNPHARMA.NS)   
6  (Volume, TATAMOTORS.NS)         (High, INFY.NS)      (High, JSWSTEEL.NS)   
7    (Close, TATASTEEL.NS)    (Open, TATASTEEL.NS)  (Volume, BAJAJ-AUTO.NS)   
8     (Open, NESTLEIND.NS)     (Volume, MARUTI.NS)     (Close, DIVISLAB.NS)   
9    (Open, TATAMOTORS.NS)         (Open, BPCL.NS)         (Low, GRASIM.NS)   

               Portfolio_4  
0      (Low, NESTLEIND.NS)  
1           (Low, BPCL.NS)  
2    

In [ ]:
portfolio_returns = pd.DataFrame(index=returns.index)
# now we are going to make the dataframe of the returns in the form of a dataframe

In [85]:
for name, stocks in portfolios.items():
    portfolio_returns[name] = returns[stocks].mean(axis=1)



In [ ]:
portfolio_returns
# these are the returns of all the portfolios.



,Portfolio_1,Portfolio_2,Portfolio_3,Portfolio_4
Date,,,,
2014-01-02,0.374622,0.631306,0.141492,-0.063961
2014-01-03,0.056602,-0.074846,-0.015638,-0.021931
2014-01-06,0.073221,-0.009770,0.009538,-0.032430
2014-01-07,-0.025572,0.191379,-0.028032,0.043258
2014-01-08,0.085087,-0.017897,0.137959,0.012382
...,...,...,...,...
2023-12-22,0.018306,0.004825,-0.075625,-0.042363
2023-12-26,-0.017860,-0.032039,0.000080,0.022568
2023-12-27,0.068043,0.012713,0.283506,0.038522


## After we have taken out the returns of the porfolios , we need to take the 